In [2]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [8]:
data = pd.read_parquet("afterextract_data.parquet").reset_index(drop = True)

In [16]:
data.head()

,id,company,origin,destination,departure,arrival,duration,vehicle_type,vehicle_class,fare,meta,insert_date
0,115386,renfe,SEVILLA,MADRID,2019-05-01 12:40:00,2019-05-01 15:23:00,2.72,AVE,Turista,Promo,{},2019-04-12 11:12:32
1,115387,renfe,SEVILLA,MADRID,2019-05-01 18:45:00,2019-05-01 21:17:00,2.53,AVE,Turista,Promo,{},2019-04-12 11:12:32
2,115389,renfe,SEVILLA,MADRID,2019-05-01 16:45:00,2019-05-01 19:17:00,2.53,AVE,Turista,Promo,{},2019-04-12 11:12:32
3,115392,renfe,SEVILLA,MADRID,2019-05-01 11:45:00,2019-05-01 14:17:00,2.53,AVE,Turista,Promo,{},2019-04-12 11:12:32
4,115394,renfe,SEVILLA,MADRID,2019-05-01 07:40:00,2019-05-01 10:05:00,2.42,AVE,Turista,Promo,{},2019-04-12 11:12:32


In [12]:
# generate train_id
data["train_id"] = data[["origin", "destination", "departure","arrival"]].sum(axis=1)

In [13]:
data["weekday"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday()
)
data["depart_month"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").month
)
data["depart_hour"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour
)
# only jan, feb, march,
data = data.loc[data["depart_month"] < 4].reset_index(drop=True)

KeyboardInterrupt: 

In [27]:
data["depart_year"] = data["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").year
)

In [28]:
data["depart_year"].value_counts()

2020    284175
Name: depart_year, dtype: int64

In [29]:
len(data["train_id"].unique())

973

In [14]:
# átvett függvény a meta, változóból való információ kinyerésére,

def extract_meta(train):
    prices_seats = pd.DataFrame()
    for i in tqdm(range(train.shape[0])):
        dic_in = json.loads(train.meta[i])
        # extract values from tree dictionary
        while type(list(dic_in.values())[0]) != float:
            res = {key: list(value) for key, value in dic_in.items()}
            dic_out = {}
            for key in res.keys():
                for value in res[key]:
                    dic_out[key + "_" + value] = dic_in[key][value]
            dic_in = dic_out

        # check for seats, if no seat its the price
        for key in list(dic_in.keys()):
            if "seats" not in key:
                if "price" not in key:
                    dic_in[key + "_price"] = dic_in[key]
                    dic_in.pop(key)

        prices_seats = pd.concat(
            [prices_seats, pd.DataFrame.from_dict(dic_in, orient="index").T]
        ).reset_index(drop=True)
    return prices_seats

In [15]:
(data.shape[0]//10000)*10000

2830000

In [16]:
#taking tons of time.
chunks = {
    i: pd.concat(
        [
            data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True),
            extract_meta(
                data.loc[i * 10000 : i * 10000 + 9999, :].reset_index(drop=True)
            ),
        ],
        axis=1,
    )
    for i in range(data.shape[0]//10000)
}

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:21<00:00, 471.30it/s]


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:31<00:00, 319.60it/s]


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:09<00:00, 1082.39it/s]


In [17]:
df = pd.concat(list(chunks.values()))

In [18]:
last_chunk = pd.concat(
    [
        data.loc[(data.shape[0]//10000)*10000:, :].reset_index(drop=True),
        extract_meta(data.loc[(data.shape[0]//10000)*10000:, :].reset_index(drop=True)),
    ]
)

100%|████████████████████████████████████████████████████████████████████████████| 7704/7704 [00:07<00:00, 1052.52it/s]


In [19]:
df = pd.concat([df,last_chunk])

In [20]:
df.shape

(2845408, 57)

In [21]:
df = df.dropna(subset=["id"]).reset_index(drop=True)

In [22]:
df.shape

(2837704, 57)

In [8]:
df["days_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    ).days
    for i in range(df.shape[0])
]

df["timedelta_till_dep"] = [
    (
        datetime.datetime.strptime(df.loc[i, "departure"][0:19], "%Y-%m-%d %H:%M:%S")
        - datetime.datetime.strptime(
            df.loc[i, "insert_date"][0:19], "%Y-%m-%d %H:%M:%S"
        )
    )
    for i in range(df.shape[0])
]

In [10]:
df.loc[:,["timedelta_till_dep","days_till_dep"]]

,timedelta_till_dep,days_till_dep
0,50 days 22:40:35,50
1,50 days 14:10:35,50
2,50 days 20:10:35,50
3,50 days 19:10:35,50
4,50 days 16:05:35,50
...,...,...
2837699,0 days 02:39:04,0
2837700,0 days 04:05:18,0
2837701,0 days 02:46:21,0
2837702,0 days 02:38:05,0


In [12]:
df.to_pickle("all_trains.pickle")

___